In [1]:
n_iterations = 100_000
n_eval_episodes = 20
#collect_episodes_per_iteration = 5
replay_buffer_capacity = 2000

batch_size = 8
collect_steps_per_iteration = 1
lr_optimizer = 2.5e-4
log_interval = 5
#n_eval_episodes = 20
eval_interval = 10
n_iterations_before_save = 5_000

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

#from tf_agents.environments import py_environment as pyenv, tf_py_environment, utils
#from tf_agents.specs import array_spec 
from tf_agents.trajectories import time_step
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.drivers.dynamic_episode_driver import DynamicEpisodeDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, Checkpointer
from tf_agents.trajectories import trajectory
from tf_agents.policies import random_tf_policy
from tf_agents.policies import policy_saver

import logging
import datetime

import os
import sys
sys.path.append(os.path.abspath(os.path.pardir))
import constants02
from constants02 import width, height, agent_state_id2str

from miner_env import MinerEnv
from tf_agents_miner_env import TFAgentsMiner

In [3]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [4]:
train_env = TFPyEnvironment(TFAgentsMiner())
eval_env = TFPyEnvironment(TFAgentsMiner())

Connected to server.
Found: map10
Found: map6
Found: map3
Found: map5
Found: map9
Found: map8
Found: map1
Found: map2
Found: map4
Found: map7
Connected to server.
Found: map10
Found: map6
Found: map3
Found: map5
Found: map9
Found: map8
Found: map1
Found: map2
Found: map4
Found: map7


In [5]:
## Creating the Deep Q-Network
preprocessing_layer = keras.layers.Lambda(
    lambda obs: tf.cast(obs, np.float32)/255.)
conv_layer_params = [(4,(3,3),1), (8,(3,3),1)]
fc_layer_params = [128, 64, 32]

q_net = QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    conv_layer_params=conv_layer_params,
    fc_layer_params=fc_layer_params,
)

In [6]:
## Creating the DQN Agent
global_step = tf.compat.v1.train.get_or_create_global_step()
optimizer = keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0, epsilon=0.00001, centered=True)

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0, # initial ε
    #decay_steps=380_000,
    decay_steps=2_000,
    end_learning_rate=0.01) # final ε

agent = DqnAgent(train_env.time_step_spec(),
                 train_env.action_spec(),
                 q_network=q_net,
                 optimizer=optimizer,
                 target_update_period=50,
                 td_errors_loss_fn=keras.losses.Huber(reduction="none"),
                 gamma=0.95, # discount factor
                 train_step_counter=global_step,
                 epsilon_greedy=lambda: epsilon_fn(global_step),
)

agent.initialize()

In [7]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [8]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

In [9]:
example_environment = TFPyEnvironment(TFAgentsMiner())

Connected to server.
Found: map10
Found: map6
Found: map3
Found: map5
Found: map9
Found: map8
Found: map1
Found: map2
Found: map4
Found: map7


In [10]:
time_step = example_environment.reset()

In [11]:
random_policy.action(time_step)

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, state=(), info=())

## Metrics and Evaluation

The most common metric used to evaluate a policy is the average return. The return is the sum of rewards obtained while running a policy in an environment for an episode. Several episodes are run, creating an average return.

The following function computes the average return of a policy, given the policy, environment, and a number of episodes.


In [12]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

Running this computation on the `random_policy` shows a baseline performance in the environment.

In [13]:
compute_avg_return(eval_env, random_policy, n_eval_episodes)

-1085.1348

## Replay Buffer

The replay buffer keeps track of data collected from the environment. This tutorial uses `tf_agents.replay_buffers.tf_uniform_replay_buffer.TFUniformReplayBuffer`, as it is the most common. 

The constructor requires the specs for the data it will be collecting. This is available from the agent using the `collect_data_spec` method. The batch size and maximum buffer length are also required.


In [14]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

For most agents, `collect_data_spec` is a named tuple called `Trajectory`, containing the specs for observations, actions, rewards, and other items.

## Data Collection

Now execute the random policy in the environment for a few steps, recording the data in the replay buffer.

In [15]:
#@test {"skip": true}
def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, steps=100)

# This loop is so common in RL, that we provide standard implementations. 
# For more details see the drivers module.
# https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers

The replay buffer is now a collection of Trajectories.

In [16]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)


dataset

<PrefetchDataset shapes: (Trajectory(step_type=(8, 2), observation=(8, 2, 9, 21, 2), action=(8, 2), policy_info=(), next_step_type=(8, 2), reward=(8, 2), discount=(8, 2)), BufferInfo(ids=(8, 2), probabilities=(8,))), types: (Trajectory(step_type=tf.int32, observation=tf.float32, action=tf.int32, policy_info=(), next_step_type=tf.int32, reward=tf.float32, discount=tf.float32), BufferInfo(ids=tf.int64, probabilities=tf.float32))>

In [17]:
iterator = iter(dataset)

print(iterator)


In [18]:
next(iterator)

(Trajectory(step_type=<tf.Tensor: shape=(8, 2), dtype=int32, numpy=
 array([[1, 1],
        [2, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 2],
        [1, 2],
        [1, 1]], dtype=int32)>, observation=<tf.Tensor: shape=(8, 2, 9, 21, 2), dtype=float32, numpy=
 array([[[[[200. ,   0. ],
           [-10. ,   0. ],
           [-10. ,   0. ],
           ...,
           [ -5.1,   0. ],
           [450. ,   0. ],
           [ -5.1,   0. ]],
 
          [[-10. ,   0. ],
           [-10. ,   0. ],
           [-20. ,   0. ],
           ...,
           [ -1. ,   0. ],
           [ -5.1,   0. ],
           [ -1. ,   0. ]],
 
          [[ -5.1,   0. ],
           [-10. ,   0. ],
           [-20. ,   0. ],
           ...,
           [150. ,   0. ],
           [-10. ,   0. ],
           [ -1. ,   0. ]],
 
          ...,
 
          [[450. ,   0. ],
           [ -5.1,   0. ],
           [ -1. ,   0. ],
           ...,
           [ -1. ,   0. ],
           [ -1. ,   0. ],
       

## Training the agent

Two things must happen during the training loop:

-   collect data from the environment
-   use that data to train the agent's neural network(s)

This example also periodicially evaluates the policy and prints the current score.

The following will take ~5 minutes to run.

In [19]:
now = datetime.datetime.now()
now_str = now.strftime("%Y%m%d-%H%M")
#script_name = __file__.split('.')[0]
script_name = "debug_empty_buffer"
save_dir = os.path.join("models", script_name)
os.makedirs(save_dir, exist_ok=True)
#logging.basicConfig(filename=os.path.join(save_path, f"log-{now_str}.txt"),level=logging.DEBUG)
checkpoint_dir = os.path.join(save_dir, 'checkpoint')
train_checkpointer = Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=500,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=global_step
)

In [20]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, n_eval_episodes)
returns = [avg_return]

for _ in range(n_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  for _ in range(collect_steps_per_iteration):
    collect_step(train_env, agent.collect_policy, replay_buffer)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0:7d}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, n_eval_episodes)
    print('step = {0:7d}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)
    if step > n_iterations_before_save:
      train_checkpointer.save(global_step)

step =       5: loss = 6.66804313659668
step =      10: loss = 176.8846893310547
step =      10: Average Return = -1032.8948974609375
step =      15: loss = 214.32777404785156
step =      20: loss = 142.0837860107422
step =      20: Average Return = -2024.08984375
step =      25: loss = 38.01915740966797
step =      30: loss = 51.15732192993164
step =      30: Average Return = -2413.525390625
step =      35: loss = 133.98533630371094
step =      40: loss = 169.17343139648438
step =      40: Average Return = -1737.4449462890625
step =      45: loss = 14.611785888671875
step =      50: loss = 137.79757690429688
step =      50: Average Return = -1716.1500244140625
step =      55: loss = 42.329166412353516
step =      60: loss = 142.25685119628906
step =      60: Average Return = -1007.7849731445312
step =      65: loss = 110.37610626220703
step =      70: loss = 11.480761528015137
step =      70: Average Return = -873.9299926757812
step =      75: loss = 46.320960998535156
step =      80:

step =     620: loss = 232.32717895507812
step =     620: Average Return = -925.2698974609375
step =     625: loss = 2.7252869606018066
step =     630: loss = 78.31094360351562
step =     630: Average Return = -2368.01025390625
step =     635: loss = 87.34254455566406
step =     640: loss = 22.151931762695312
step =     640: Average Return = -3135.230224609375
step =     645: loss = 33.90730285644531
step =     650: loss = 48.997737884521484
step =     650: Average Return = -3719.344970703125
step =     655: loss = 64.64601135253906
step =     660: loss = 76.58393096923828
step =     660: Average Return = -1021.6300048828125
step =     665: loss = 9.14881706237793
step =     670: loss = 11.764723777770996
step =     670: Average Return = -966.9549560546875
step =     675: loss = 26.412885665893555
step =     680: loss = 178.109130859375
step =     680: Average Return = -863.5250244140625
step =     685: loss = 33.536956787109375
step =     690: loss = 58.86405944824219
step =     690: 

step =    1230: Average Return = -958.2799682617188
step =    1235: loss = 49.53599548339844
step =    1240: loss = 19.707843780517578
step =    1240: Average Return = -859.0549926757812
step =    1245: loss = 38.04528045654297
step =    1250: loss = 129.62229919433594
step =    1250: Average Return = -1008.5598754882812
step =    1255: loss = 108.95425415039062
step =    1260: loss = 127.24549865722656
step =    1260: Average Return = -925.64501953125
step =    1265: loss = 233.85557556152344
step =    1270: loss = 147.97225952148438
step =    1270: Average Return = -990.4000244140625
step =    1275: loss = 277.5623474121094
step =    1280: loss = 131.21063232421875
step =    1280: Average Return = -1022.43994140625
step =    1285: loss = 169.05831909179688
step =    1290: loss = 129.3597869873047
step =    1290: Average Return = -1085.574951171875
step =    1295: loss = 47.12593460083008
step =    1300: loss = 132.98890686035156
step =    1300: Average Return = -1047.64501953125
step

step =    1840: Average Return = -747.0449829101562
step =    1845: loss = 143.8124542236328
step =    1850: loss = 25.23461151123047
step =    1850: Average Return = -737.8599243164062
step =    1855: loss = 163.92221069335938
step =    1860: loss = 38.531494140625
step =    1860: Average Return = -996.0549926757812
step =    1865: loss = 20.04701805114746
step =    1870: loss = 24.940269470214844
step =    1870: Average Return = -775.43994140625
step =    1875: loss = 68.03485107421875
step =    1880: loss = 48.96816635131836
step =    1880: Average Return = -954.7200317382812
step =    1885: loss = 14.958769798278809
step =    1890: loss = 61.20819091796875
step =    1890: Average Return = -874.9749755859375
step =    1895: loss = 20.367542266845703
step =    1900: loss = 9.539169311523438
step =    1900: Average Return = -887.4700317382812
step =    1905: loss = 5.060938835144043
step =    1910: loss = 44.3570442199707
step =    1910: Average Return = -846.9150390625
step =    1915

step =    2460: loss = 5.700041770935059
step =    2460: Average Return = -865.4150390625
step =    2465: loss = 25.55169677734375
step =    2470: loss = 216.1776885986328
step =    2470: Average Return = -800.1299438476562
step =    2475: loss = 146.95982360839844
step =    2480: loss = 154.60372924804688
step =    2480: Average Return = -803.1700439453125
step =    2485: loss = 39.844871520996094
step =    2490: loss = 8.838882446289062
step =    2490: Average Return = -1415.760009765625
step =    2495: loss = 136.8541717529297
step =    2500: loss = 36.62031555175781
step =    2500: Average Return = -861.93505859375
step =    2505: loss = 61.643455505371094
step =    2510: loss = 56.1763801574707
step =    2510: Average Return = -3642.520263671875
step =    2515: loss = 112.38270568847656
step =    2520: loss = 15.087356567382812
step =    2520: Average Return = -1353.2198486328125
step =    2525: loss = 154.3855743408203
step =    2530: loss = 141.46937561035156
step =    2530: Ave

...

step =   27590: Average Return = -1049.869873046875
step =   27595: loss = 156.31015014648438
step =   27600: loss = 122.48418426513672
step =   27600: Average Return = -988.3250122070312
step =   27605: loss = 102.72280883789062
step =   27610: loss = 108.1952133178711
step =   27610: Average Return = -742.7249755859375
step =   27615: loss = 130.79200744628906
step =   27620: loss = 45.91440200805664
step =   27620: Average Return = -728.8999633789062
step =   27625: loss = 139.20802307128906
step =   27630: loss = 15.005265235900879
step =   27630: Average Return = -743.6749267578125
step =   27635: loss = 12.882209777832031
step =   27640: loss = 102.17779541015625
step =   27640: Average Return = -693.449951171875
step =   27645: loss = 104.04618835449219
step =   27650: loss = 41.24293518066406
step =   27650: Average Return = -929.9849853515625
step =   27655: loss = 136.2050018310547
step =   27660: loss = 16.825620651245117
step =   27660: Average Return = -912.4100341796875
s

step =   28200: Average Return = -901.8848876953125
step =   28205: loss = 24.153915405273438
step =   28210: loss = 126.50576782226562
step =   28210: Average Return = -988.0499267578125
step =   28215: loss = 14.804571151733398
step =   28220: loss = 91.43391418457031
step =   28220: Average Return = -688.4150390625
step =   28225: loss = 21.224895477294922
step =   28230: loss = 64.72725677490234
step =   28230: Average Return = -974.8499145507812
step =   28235: loss = 17.58477210998535
step =   28240: loss = 15.43202018737793
step =   28240: Average Return = -849.0599365234375
step =   28245: loss = 100.89723205566406
step =   28250: loss = 58.41675567626953
step =   28250: Average Return = -884.1549682617188
step =   28255: loss = 16.261459350585938
step =   28260: loss = 95.84001922607422
step =   28260: Average Return = -848.8349609375
step =   28265: loss = 265.79168701171875
step =   28270: loss = 12.893369674682617
step =   28270: Average Return = -999.3150634765625
step =  

step =   28810: Average Return = -886.9699096679688
step =   28815: loss = 222.35546875
step =   28820: loss = 33.29438018798828
step =   28820: Average Return = -1126.81494140625
step =   28825: loss = 62.23832702636719
step =   28830: loss = 122.46232604980469
step =   28830: Average Return = -1016.2899169921875
step =   28835: loss = 9.283279418945312
step =   28840: loss = 274.17822265625
step =   28840: Average Return = -898.0900268554688
step =   28845: loss = 21.137292861938477
step =   28850: loss = 84.2211685180664
step =   28850: Average Return = -821.3899536132812
step =   28855: loss = 118.03887176513672
step =   28860: loss = 236.91854858398438
step =   28860: Average Return = -734.6799926757812
step =   28865: loss = 93.28922271728516
step =   28870: loss = 129.28988647460938
step =   28870: Average Return = -816.0748901367188
step =   28875: loss = 190.80911254882812
step =   28880: loss = 59.43940734863281
step =   28880: Average Return = -734.324951171875
step =   2888

step =   29425: loss = 19.645402908325195
step =   29430: loss = 148.50357055664062
step =   29430: Average Return = -960.56982421875
step =   29435: loss = 8.708415031433105
step =   29440: loss = 20.158044815063477
step =   29440: Average Return = -1063.3099365234375
step =   29445: loss = 191.59690856933594
step =   29450: loss = 100.42283630371094
step =   29450: Average Return = -911.1649169921875
step =   29455: loss = 21.35818862915039
step =   29460: loss = 166.92904663085938
step =   29460: Average Return = -667.5399169921875
step =   29465: loss = 146.6412811279297
step =   29470: loss = 32.16246032714844
step =   29470: Average Return = -775.2449340820312
step =   29475: loss = 46.824710845947266
step =   29480: loss = 118.46725463867188
step =   29480: Average Return = -803.2550048828125
step =   29485: loss = 39.310791015625
step =   29490: loss = 83.67474365234375
step =   29490: Average Return = -852.4849853515625
step =   29495: loss = 86.46823120117188
step =   29500: 

KeyboardInterrupt: 

## Visualization


### Plots

Use `matplotlib.pyplot` to chart how the policy improved during training.

One iteration of `Cartpole-v0` consists of 200 time steps. The environment gives a reward of `+1` for each step the pole stays up, so the maximum return for one episode is 200. The charts shows the return increasing towards that maximum each time it is evaluated during training. (It may be a little unstable and not increase monotonically each time.)

### Videos

Charts are nice. But more exciting is seeing an agent actually performing a task in an environment. 

First, create a function to embed videos in the notebook.

Now iterate through a few episodes of the Cartpole game with the agent. The underlying Python environment (the one "inside" the TensorFlow environment wrapper) provides a `render()` method, which outputs an image of the environment state. These can be collected into a video.

For fun, compare the trained agent (above) to an agent moving randomly. (It does not do as well.)